In [33]:
import os
import glob
import json
from PIL import Image
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from datetime import datetime
import timm


# ---- Paths (adjust if needed) ----
DATA_INPUT = '/kaggle/input'
DATASET_NAME = 'spurious-correlation'
DATASET_ROOT = os.path.join(DATA_INPUT, DATASET_NAME)

# Candidate places where the Oxford data may live (handles extra nested dirs)
CANDIDATE_OXFORD_ROOTS = [
    os.path.join(DATASET_ROOT, 'oxford-iiit-pet'),
    os.path.join(DATASET_ROOT, 'oxford-iiit-pet', 'oxford-iiit-pet'),
    os.path.join(DATASET_ROOT, 'oxford_iiit_pet'),
]

In [34]:
def find_oxford_root():
    for root in CANDIDATE_OXFORD_ROOTS:
        images_dir = os.path.join(root, 'images')
        ann_dir = os.path.join(root, 'annotations')
        # also accept nested 'annotations/annotations'
        alt_ann_dir = os.path.join(root, 'annotations', 'annotations')
        if os.path.isdir(images_dir) and (os.path.isdir(ann_dir) or os.path.isdir(alt_ann_dir)):
            return root
    raise FileNotFoundError("Couldn't find oxford-iiit-pet root in expected candidate paths. "
                            f"Checked: {CANDIDATE_OXFORD_ROOTS}")

In [35]:
OXFORD_ROOT = find_oxford_root()
print("Using Oxford root:", OXFORD_ROOT)

Using Oxford root: /kaggle/input/spurious-correlation/oxford-iiit-pet/oxford-iiit-pet


In [36]:
# canonical images/ and annotations/ (resolve nested annotations)
IMAGES_DIR = os.path.join(OXFORD_ROOT, 'images')
ANNOTATIONS_DIR = os.path.join(OXFORD_ROOT, 'annotations')

if not os.path.isdir(ANNOTATIONS_DIR):
    # fallback to nested 'annotations/annotations'
    nested = os.path.join(OXFORD_ROOT, 'annotations', 'annotations')
    if os.path.isdir(nested):
        ANNOTATIONS_DIR = nested

print("Images dir:", IMAGES_DIR)
print("Annotations dir:", ANNOTATIONS_DIR)

Images dir: /kaggle/input/spurious-correlation/oxford-iiit-pet/oxford-iiit-pet/images
Annotations dir: /kaggle/input/spurious-correlation/oxford-iiit-pet/oxford-iiit-pet/annotations


In [37]:
# Try several likely locations for trainval.txt
possible_trainval = [
    os.path.join(ANNOTATIONS_DIR, 'trainval.txt'),
    os.path.join(OXFORD_ROOT, 'trainval.txt'),
    os.path.join(OXFORD_ROOT, 'annotations', 'trainval.txt'),
    os.path.join(OXFORD_ROOT, 'annotations', 'annotations', 'trainval.txt'),
]

trainval_path = None

# First check direct candidate paths
for p in possible_trainval:
    if os.path.isfile(p):
        trainval_path = p
        break

# If still None, try recursive glob search
if trainval_path is None:
    matches = glob.glob(os.path.join(OXFORD_ROOT, '**', 'trainval.txt'), recursive=True)
    if len(matches) > 0:
        trainval_path = matches[0]

# Final validation
if trainval_path is None:
    raise FileNotFoundError(
        "trainval.txt not found.\n"
        "Tried the following paths:\n" +
        "\n".join(possible_trainval)
    )

print("✅ Found trainval.txt at:", trainval_path)

✅ Found trainval.txt at: /kaggle/input/spurious-correlation/oxford-iiit-pet/oxford-iiit-pet/annotations/annotations/trainval.txt


In [38]:
# Fix images directory if an extra nested 'images' folder exists
if os.path.isdir(os.path.join(IMAGES_DIR, 'images')):
    IMAGES_DIR = os.path.join(IMAGES_DIR, 'images')

# Fix annotations directory similarly
if os.path.isdir(os.path.join(ANNOTATIONS_DIR, 'annotations')):
    ANNOTATIONS_DIR = os.path.join(ANNOTATIONS_DIR, 'annotations')

if os.path.isdir(os.path.join(ANNOTATIONS_DIR, 'annotations')):
    ANNOTATIONS_DIR = os.path.join(ANNOTATIONS_DIR, 'annotations')

print("Using IMAGES_DIR:", IMAGES_DIR)
print("Using ANNOTATIONS_DIR:", ANNOTATIONS_DIR)


Using IMAGES_DIR: /kaggle/input/spurious-correlation/oxford-iiit-pet/oxford-iiit-pet/images/images
Using ANNOTATIONS_DIR: /kaggle/input/spurious-correlation/oxford-iiit-pet/oxford-iiit-pet/annotations/annotations


In [39]:
file_list = []
with open(trainval_path, 'r') as f:
    lines = [ln.strip() for ln in f.readlines() if ln.strip()]
for ln in lines:
    tokens = ln.split()
    if len(tokens) == 0:
        continue
    img_name = tokens[0]
    class_id = int(tokens[1]) - 1 if len(tokens) > 1 else None
    image_path = os.path.join(IMAGES_DIR, img_name + '.jpg')
    # mask / trimap path lives under "annotations/trimaps" and is named <img_name>.png
    trimap_path = os.path.join(ANNOTATIONS_DIR, 'trimaps', img_name + '.png')
    # fallback if extension is .jpg or .jpeg or .PNG etc.
    if not os.path.isfile(image_path):
        alt = None
        for ext in ['.jpg', '.jpeg', '.png']:
            altp = os.path.join(IMAGES_DIR, img_name + ext)
            if os.path.isfile(altp):
                alt = altp; break
        if alt is None:
            raise FileNotFoundError(f"Image {img_name} not found under {IMAGES_DIR}")
        image_path = alt
    if not os.path.isfile(trimap_path):
        # try alternate locations (some datasets use 'annotations' root directly)
        alt_trim = os.path.join(ANNOTATIONS_DIR, img_name + '.png')
        if os.path.isfile(alt_trim):
            trimap_path = alt_trim
        else:
            # mask might be missing for some datasets; we won't crash but set None
            trimap_path = None

    file_list.append({
        'image_name': img_name,
        'image_path': image_path,
        'trimap_path': trimap_path,
        'class_id': class_id
    })

print(f"Total examples parsed from trainval.txt: {len(file_list)}")

Total examples parsed from trainval.txt: 3680


In [40]:
# Load your split metadata (expects indices relative to the file_list ordering)
SPLIT_METADATA = os.path.join(DATASET_ROOT, 'split_metadata.json')
if not os.path.isfile(SPLIT_METADATA):
    raise FileNotFoundError("split_metadata.json not found at: " + SPLIT_METADATA)

with open(SPLIT_METADATA, 'r') as f:
    split_info = json.load(f)
train_indices = split_info['train_indices']
val_indices = split_info['val_indices']
print("Split sizes -> train:", len(train_indices), "val:", len(val_indices))
assert max(train_indices) < len(file_list) and max(val_indices) < len(file_list), "Split indices exceed dataset length"
# Cell 4: Load Split Metadata
print("Loading split metadata...")

with open(SPLIT_METADATA, 'r') as f:
    split_info = json.load(f)

train_indices = split_info['train_indices']
val_indices = split_info['val_indices']

print(f"✓ Split loaded:")
print(f"  Train indices: {len(train_indices)}")
print(f"  Val indices: {len(val_indices)}")
print(f"  Total: {len(train_indices) + len(val_indices)}")

# Verify indices are valid
assert max(train_indices) < len(file_list), "Invalid train index"
assert max(val_indices) < len(file_list), "Invalid val index"
assert len(set(train_indices) & set(val_indices)) == 0, "Overlap between train and val"
print("✓ Split indices validated")

Split sizes -> train: 2944 val: 736
Loading split metadata...
✓ Split loaded:
  Train indices: 2944
  Val indices: 736
  Total: 3680
✓ Split indices validated


In [41]:
# ---- Transforms (re-using yours) ----
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
IMG_SIZE = 224

train_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.RandomRotation(degrees=10),
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

In [42]:
# ---- Custom dataset class (does not rely on torchvision OxfordIIITPet) ----
class OxfordPetFileList(Dataset):
    def __init__(self, file_list, indices, transform=None, include_metadata=False):
        """
        file_list: list of dicts with keys 'image_path','trimap_path','class_id'
        indices: list of ints referencing positions in file_list
        """
        self._file_list = file_list
        self.indices = indices
        self.transform = transform
        self.include_metadata = include_metadata

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        actual_idx = self.indices[idx]
        rec = self._file_list[actual_idx]

        # Load image
        img = Image.open(rec['image_path']).convert('RGB')

        # Load mask/trimap if exists (keep as single channel)
        mask = None
        if rec['trimap_path'] and os.path.isfile(rec['trimap_path']):
            mask = Image.open(rec['trimap_path'])
            # convert to 'L' to ensure single channel numeric values
            mask = mask.convert('L')

        # Save original metadata if requested
        original_image = np.array(img, dtype=np.uint8) if self.include_metadata else None
        original_mask  = np.array(mask, dtype=np.uint8) if (self.include_metadata and mask is not None) else None

        # Apply transforms (to the image only)
        if self.transform:
            img_t = self.transform(img)
        else:
            img_t = T.ToTensor()(img)

        label = rec['class_id']
        sample = {
            'image': img_t,
            'label': label,
            'index': actual_idx
        }
        if self.include_metadata:
            sample['original_image'] = original_image
            sample['original_mask'] = original_mask

        return sample

print("✓ Custom file-list dataset class ready")

✓ Custom file-list dataset class ready


In [43]:
# ---- Create dataset & dataloaders ----
batch_size = 32
num_workers = 4

train_dataset = OxfordPetFileList(file_list=file_list, indices=train_indices, transform=train_transform, include_metadata=False)
val_dataset   = OxfordPetFileList(file_list=file_list, indices=val_indices, transform=val_transform, include_metadata=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

print("Loaders created:")
print(" - train batches:", len(train_loader))
print(" - val batches:  ", len(val_loader))

# ---- Quick sanity check: iterate one batch ----
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bs = next(iter(train_loader))
print("Sanity batch keys:", list(bs.keys()))
print("Image tensor shape:", bs['image'].shape)
print("Label tensor sample:", bs['label'][:8])

Loaders created:
 - train batches: 92
 - val batches:   23
Sanity batch keys: ['image', 'label', 'index']
Image tensor shape: torch.Size([32, 3, 224, 224])
Label tensor sample: tensor([19, 18, 31, 28, 12, 17, 15, 27])


In [44]:
# Cell 1: RETRAIN - ResNet-50

print("\n" + "="*70)
print("TRAINING: ResNet-50")
print("="*70)

model_key = 'resnet50'
model_name = 'ResNet-50'

LEARNING_RATE = 3e-4      # You can adjust this based on your model
WEIGHT_DECAY = 1e-2     # Common default value
EPOCHS = 30 
BATCH_SIZE = 64
# Create model
model = models.resnet50(weights='DEFAULT')
model.fc = nn.Linear(model.fc.in_features, 37)
model = model.to(device)

print(f"✓ {model_name} loaded")

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
criterion = nn.CrossEntropyLoss()

# Training
history = {
    'epoch': [],
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0.0
start_time = datetime.now()

for epoch in range(EPOCHS):
    # Train
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [TRAIN]', leave=False)
    for batch in pbar:
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)
        
        pbar.set_postfix({'loss': f'{train_loss/(train_total//BATCH_SIZE+1):.4f}', 
                         'acc': f'{100*train_correct/train_total:.1f}%'})
    
    epoch_train_loss = train_loss / len(train_loader)
    epoch_train_acc = train_correct / train_total
    
    # Validate
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [VAL]', leave=False)
    with torch.no_grad():
        for batch in pbar:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
            
            pbar.set_postfix({'loss': f'{val_loss/(val_total//BATCH_SIZE+1):.4f}',
                             'acc': f'{100*val_correct/val_total:.1f}%'})
    
    epoch_val_loss = val_loss / len(val_loader)
    epoch_val_acc = val_correct / val_total
    
    scheduler.step()
    
    history['epoch'].append(epoch + 1)
    history['train_loss'].append(epoch_train_loss)
    history['train_acc'].append(epoch_train_acc)
    history['val_loss'].append(epoch_val_loss)
    history['val_acc'].append(epoch_val_acc)
    
    print(f"Epoch {epoch+1}/{EPOCHS}: Train Acc={epoch_train_acc*100:.1f}% | Val Acc={epoch_val_acc*100:.1f}%")
    
    # Save best
    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        checkpoint_dir = f'/kaggle/working/checkpoints/{model_key}'
        os.makedirs(checkpoint_dir, exist_ok=True)
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_acc': best_val_acc,
            'history': history
        }, f'{checkpoint_dir}/best.pth')
        
        print(f"  ✓ Best model saved (Val Acc: {best_val_acc*100:.2f}%)")

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

print(f"\n{'='*70}")
print(f"✅ ResNet-50 TRAINING COMPLETE")
print(f"  Time: {total_time/60:.1f} minutes")
print(f"  Best Val Acc: {best_val_acc*100:.2f}%")
print(f"{'='*70}\n")

torch.cuda.empty_cache()



TRAINING: ResNet-50
✓ ResNet-50 loaded


Epoch 1/30: Train Acc=69.7% | Val Acc=82.5%
  ✓ Best model saved (Val Acc: 82.47%)


Epoch 2/30: Train Acc=88.2% | Val Acc=88.5%
  ✓ Best model saved (Val Acc: 88.45%)


Epoch 3/30: Train Acc=93.0% | Val Acc=87.5%


Epoch 4/30: Train Acc=93.9% | Val Acc=86.0%


...

Epoch 26/30: Train Acc=99.9% | Val Acc=93.5%
  ✓ Best model saved (Val Acc: 93.48%)


Epoch 27/30: Train Acc=100.0% | Val Acc=93.3%


Epoch 28/30: Train Acc=100.0% | Val Acc=93.3%


Epoch 29/30: Train Acc=100.0% | Val Acc=92.8%


Epoch 30/30: Train Acc=100.0% | Val Acc=93.6%
  ✓ Best model saved (Val Acc: 93.61%)

✅ ResNet-50 TRAINING COMPLETE
  Time: 10.5 minutes
  Best Val Acc: 93.61%



In [46]:
# Cell 2: RETRAIN - Swin-Tiny (FIXED)

print("\n" + "="*70)
print("TRAINING: Swin-Tiny")
print("="*70)

model_key = 'swin_tiny'
model_name = 'Swin-Tiny'

LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-2
EPOCHS = 30
BATCH_SIZE = 48

# Create model
model = timm.create_model(
    'swin_tiny_patch4_window7_224',
    pretrained=True,
    num_classes=37  # ensures correct classification head
).to(device)

print(f"✓ {model_name} loaded")

# Optimizer, scheduler, loss
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
criterion = nn.CrossEntropyLoss()

# Training history
history = {
    'epoch': [],
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0.0
best_epoch = 0
start_time = datetime.now()

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [TRAIN]', leave=False)
    for batch in pbar:
        images = batch['image'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(images)

        # 🔧 Fix for 4D output (apply global avg pooling if needed)
        if outputs.dim() == 4:
            outputs = outputs.mean(dim=[2, 3])

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

        pbar.set_postfix({
            'loss': f'{train_loss/(train_total//BATCH_SIZE+1):.4f}',
            'acc': f'{100*train_correct/train_total:.1f}%'
        })

    epoch_train_loss = train_loss / len(train_loader)
    epoch_train_acc = train_correct / train_total

    # -------------------- Validation --------------------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [VAL]', leave=False)
    with torch.no_grad():
        for batch in pbar:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            outputs = model(images)

            # 🔧 again ensure output is 2D
            if outputs.dim() == 4:
                outputs = outputs.mean(dim=[2, 3])

            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

            pbar.set_postfix({
                'loss': f'{val_loss/(val_total//BATCH_SIZE+1):.4f}',
                'acc': f'{100*val_correct/val_total:.1f}%'
            })

    epoch_val_loss = val_loss / len(val_loader)
    epoch_val_acc = val_correct / val_total
    scheduler.step()

    history['epoch'].append(epoch + 1)
    history['train_loss'].append(epoch_train_loss)
    history['train_acc'].append(epoch_train_acc)
    history['val_loss'].append(epoch_val_loss)
    history['val_acc'].append(epoch_val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS}: "
          f"Train Acc={epoch_train_acc*100:.1f}% | Val Acc={epoch_val_acc*100:.1f}%")

    # Save best model
    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        best_epoch = epoch + 1
        checkpoint_dir = f'/kaggle/working/checkpoints/{model_key}'
        os.makedirs(checkpoint_dir, exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_acc': best_val_acc,
            'history': history
        }, f'{checkpoint_dir}/best.pth')
        print(f"  ✓ Best model saved (Val Acc: {best_val_acc*100:.2f}%)")

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

print(f"\n{'='*70}")
print(f"✅ Swin-Tiny TRAINING COMPLETE")
print(f"  Time: {total_time/60:.1f} minutes")
print(f"  Best Epoch: {best_epoch}")
print(f"  Best Val Acc: {best_val_acc*100:.2f}%")
print(f"{'='*70}\n")

# Summary Table
import pandas as pd
summary = pd.DataFrame(history)
display(summary.tail())

torch.cuda.empty_cache()


TRAINING: Swin-Tiny
✓ Swin-Tiny loaded


Epoch 1/30: Train Acc=68.3% | Val Acc=78.5%
  ✓ Best model saved (Val Acc: 78.53%)


Epoch 2/30: Train Acc=84.4% | Val Acc=84.2%
  ✓ Best model saved (Val Acc: 84.24%)


Epoch 3/30: Train Acc=89.1% | Val Acc=85.7%
  ✓ Best model saved (Val Acc: 85.73%)


Epoch 4/30: Train Acc=92.2% | Val Acc=87.6%
  ✓ Best model saved (Val Acc: 87.64%)


...

Epoch 27/30: Train Acc=99.8% | Val Acc=91.0%


Epoch 28/30: Train Acc=99.9% | Val Acc=91.3%


Epoch 29/30: Train Acc=100.0% | Val Acc=91.3%


Epoch 30/30: Train Acc=99.8% | Val Acc=91.3%

✅ Swin-Tiny TRAINING COMPLETE
  Time: 13.3 minutes
  Best Epoch: 24
  Best Val Acc: 91.85%



,epoch,train_loss,train_acc,val_loss,val_acc
25,26,0.006181,0.998981,0.364574,0.918478
26,27,0.005690,0.997962,0.364238,0.910326
27,28,0.004878,0.999321,0.357687,0.913043
28,29,0.001845,1.000000,0.357777,0.913043
29,30,0.005150,0.998302,0.355736,0.913043


In [47]:
# Cell 3: RETRAIN - DeiT-Small

print("\n" + "="*70)
print("TRAINING: DeiT-Small")
print("="*70)

model_key = 'deit_small'
model_name = 'DeiT-Small'

LEARNING_RATE = 3e-4      # You can adjust this based on your model
WEIGHT_DECAY = 1e-2       # Common default value
EPOCHS = 30 
BATCH_SIZE = 48

# Create model
model = timm.create_model('deit_small_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 37)
model = model.to(device)

print(f"✓ {model_name} loaded")

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
criterion = nn.CrossEntropyLoss()

# Training
history = {
    'epoch': [],
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0.0
start_time = datetime.now()

for epoch in range(EPOCHS):
    # Train
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [TRAIN]', leave=False)
    for batch in pbar:
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)
        
        pbar.set_postfix({'loss': f'{train_loss/(train_total//BATCH_SIZE+1):.4f}', 
                         'acc': f'{100*train_correct/train_total:.1f}%'})
    
    epoch_train_loss = train_loss / len(train_loader)
    epoch_train_acc = train_correct / train_total
    
    # Validate
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [VAL]', leave=False)
    with torch.no_grad():
        for batch in pbar:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
            
            pbar.set_postfix({'loss': f'{val_loss/(val_total//BATCH_SIZE+1):.4f}',
                             'acc': f'{100*val_correct/val_total:.1f}%'})
    
    epoch_val_loss = val_loss / len(val_loader)
    epoch_val_acc = val_correct / val_total
    
    scheduler.step()
    
    history['epoch'].append(epoch + 1)
    history['train_loss'].append(epoch_train_loss)
    history['train_acc'].append(epoch_train_acc)
    history['val_loss'].append(epoch_val_loss)
    history['val_acc'].append(epoch_val_acc)
    
    print(f"Epoch {epoch+1}/{EPOCHS}: Train Acc={epoch_train_acc*100:.1f}% | Val Acc={epoch_val_acc*100:.1f}%")
    
    # Save best
    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        checkpoint_dir = f'/kaggle/working/checkpoints/{model_key}'
        os.makedirs(checkpoint_dir, exist_ok=True)
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_acc': best_val_acc,
            'history': history
        }, f'{checkpoint_dir}/best.pth')
        
        print(f"  ✓ Best model saved (Val Acc: {best_val_acc*100:.2f}%)")

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

print(f"\n{'='*70}")
print(f"✅ DeiT-Small TRAINING COMPLETE")
print(f"  Time: {total_time/60:.1f} minutes")
print(f"  Best Val Acc: {best_val_acc*100:.2f}%")
print(f"{'='*70}\n")

torch.cuda.empty_cache()



TRAINING: DeiT-Small


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

✓ DeiT-Small loaded


Epoch 1/30: Train Acc=69.1% | Val Acc=84.2%
  ✓ Best model saved (Val Acc: 84.24%)


Epoch 2/30: Train Acc=88.0% | Val Acc=80.4%


...

Epoch 26/30: Train Acc=100.0% | Val Acc=90.2%


Epoch 27/30: Train Acc=100.0% | Val Acc=90.2%


Epoch 28/30: Train Acc=100.0% | Val Acc=90.2%


Epoch 29/30: Train Acc=100.0% | Val Acc=90.8%
  ✓ Best model saved (Val Acc: 90.76%)


Epoch 30/30: Train Acc=100.0% | Val Acc=90.8%

✅ DeiT-Small TRAINING COMPLETE
  Time: 10.7 minutes
  Best Val Acc: 90.76%



In [ ]:
# Cell 4: Verify Training Completed

print("\n" + "="*70)
print("✅ TRAINING COMPLETE - VERIFYING CHECKPOINTS")
print("="*70)

models_to_check = ['resnet50', 'swin_tiny', 'deit_small']

for model_key in models_to_check:
    checkpoint_path = f'/kaggle/working/checkpoints/{model_key}/best.pth'
    
    if os.path.exists(checkpoint_path):
        size = os.path.getsize(checkpoint_path) / 1e6
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        best_acc = checkpoint.get('best_val_acc', 'N/A')
        
        print(f"✓ {model_key}")
        print(f"  Path: {checkpoint_path}")
        print(f"  Size: {size:.1f} MB")
        print(f"  Best Acc: {best_acc*100:.2f}%" if isinstance(best_acc, float) else f"  Best Acc: {best_acc}")
    else:
        print(f"✗ {model_key} - NOT FOUND")

print("\n" + "="*70)
print("Ready for Day 5: Grad-CAM, FAR, and ∆Acc computation!")
print("="*70)


In [ ]:
from matplotlib.pyplot import py
# Cell 15: Plot Training Curves
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (model_key, result) in enumerate(all_results.items()):
    ax = axes[idx]
    epochs = result['history']['epoch']
    
    # Plot training and validation accuracy
    ax.plot(epochs, np.array(result['history']['train_acc'])*100, 
            label='Train', marker='o', markersize=3, linewidth=1.5, color='blue')
    ax.plot(epochs, np.array(result['history']['val_acc'])*100, 
            label='Val', marker='s', markersize=3, linewidth=1.5, color='red')
    
    # Mark best epoch
    best_epoch = np.argmax(result['history']['val_acc']) + 1
    best_acc = result['history']['val_acc'][best_epoch - 1]
    ax.axvline(x=best_epoch, color='green', linestyle='--', alpha=0.5, linewidth=1.5)
    
    # Labels and title
    ax.set_title(f"{result['name']}\nBest: {best_acc*100:.1f}% @ Epoch {best_epoch}", 
                 fontsize=11, fontweight='bold')
    ax.set_xlabel('Epoch', fontsize=10)
    ax.set_ylabel('Accuracy (%)', fontsize=10)
    ax.set_ylim([0, 100])
    ax.legend(fontsize=9, loc='lower right')
    ax.grid(True, alpha=0.3)

plt.suptitle('Baseline Model Training Curves\nResNet-50 vs Swin-Tiny vs DeiT-Small vs ConvNeXt-Tiny', 
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('/kaggle/working/results/training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training curves saved")


In [ ]:
# Cell 16: Save Detailed Results
import json

print("Saving training histories...\n")

# Save each model's history as CSV
for model_key, result in all_results.items():
    history_df = pd.DataFrame({
        'epoch': result['history']['epoch'],
        'train_loss': result['history']['train_loss'],
        'train_acc': result['history']['train_acc'],
        'val_loss': result['history']['val_loss'],
        'val_acc': result['history']['val_acc'],
        'lr': result['history']['lr']
    })
    
    history_path = f'/kaggle/working/results/{model_key}_history.csv'
    history_df.to_csv(history_path, index=False)
    print(f"✓ Saved: {model_key}_history.csv")

# Save summary JSON
summary = {
    'training_date': datetime.now().isoformat(),
    'total_epochs': EPOCHS,
    'learning_rate': LEARNING_RATE,
    'weight_decay': WEIGHT_DECAY,
    'models': {}
}

for model_key, result in all_results.items():
    best_epoch = np.argmax(result['history']['val_acc']) + 1
    summary['models'][model_key] = {
        'name': result['name'],
        'best_val_acc': float(result['best_acc']),
        'best_epoch': int(best_epoch),
        'final_train_acc': float(result['history']['train_acc'][-1]),
        'final_val_acc': float(result['history']['val_acc'][-1])
    }

with open('/kaggle/working/results/summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("\n✓ Saved: summary.json")


In [ ]:
# Cell 17: Training Complete Summary
print("\n" + "="*70)
print("✅ TRAINING PIPELINE COMPLETE")
print("="*70)

print("\nResults saved to /kaggle/working/results/:")
for file in sorted(os.listdir('/kaggle/working/results/')):
    file_path = f'/kaggle/working/results/{file}'
    file_size = os.path.getsize(file_path) / 1e6
    print(f"  ✓ {file} ({file_size:.1f} MB)")

print("\nModel checkpoints saved to /kaggle/working/checkpoints/:")
for model in sorted(os.listdir('/kaggle/working/checkpoints/')):
    ckpt_path = f'/kaggle/working/checkpoints/{model}/best.pth'
    if os.path.exists(ckpt_path):
        ckpt_size = os.path.getsize(ckpt_path) / 1e6
        print(f"  ✓ {model}/best.pth ({ckpt_size:.1f} MB)")

print("\n" + "="*70)
print("KEY RESULTS:")
print("="*70)
for model_key, result in all_results.items():
    best_epoch = np.argmax(result['history']['val_acc']) + 1
    print(f"\n{result['name']}:")
    print(f"  Best validation accuracy: {result['best_acc']*100:.2f}%")
    print(f"  Best epoch: {best_epoch}")
    print(f"  Final train accuracy: {result['history']['train_acc'][-1]*100:.2f}%")



In [ ]:
# Cell: Check for Existing Checkpoints

import os

checkpoint_dir = '/kaggle/working/checkpoints'

if os.path.exists(checkpoint_dir):
    print("✓ Checkpoints directory exists!\n")
    
    models_found = {}
    for model_name in ['resnet50', 'swin_tiny', 'deit_small', 'convnext_tiny']:
        model_dir = os.path.join(checkpoint_dir, model_name)
        if os.path.exists(model_dir):
            best_pth = os.path.join(model_dir, 'best.pth')
            if os.path.exists(best_pth):
                size = os.path.getsize(best_pth) / 1e6  # MB
                models_found[model_name] = size
                print(f"✓ {model_name}: {size:.1f} MB")
            else:
                print(f"✗ {model_name}: Directory exists but no best.pth")
        else:
            print(f"✗ {model_name}: Directory not found")
    
    if len(models_found) == 4:
        print(f"\n✅ ALL 4 MODELS FOUND! ({sum(models_found.values()):.0f} MB total)")
        print("   You can download and continue!")
    else:
        print(f"\n⚠️  Only {len(models_found)}/4 models found - retrain needed")
else:
    print("❌ Checkpoints directory does not exist - need to retrain")


In [ ]:
# Cell: Grad-CAM Analysis for All 3 Models (ResNet-50, Swin-Tiny, DeiT-Small)

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from tqdm import tqdm
import pandas as pd
import os

print("\n" + "="*80)
print("GRAD-CAM ANALYSIS - ALL 3 MODELS")
print("="*80)

# ============================================================================
# HELPER FUNCTION: COMPUTE FAR (Foreground Attention Ratio)
# ============================================================================

def compute_far(cam, mask):
    """
    Compute Foreground Attention Ratio
    
    Args:
        cam: Grad-CAM activation map (normalized 0-1)
        mask: Binary segmentation mask (1=foreground, 0=background)
    
    Returns:
        FAR: Fraction of attention on foreground (0-1)
    """
    foreground_attention = cam[mask == 1].sum()
    total_attention = cam.sum()
    return float(foreground_attention / total_attention) if total_attention > 0 else 0.0

# ============================================================================
# GET TARGET LAYERS FOR EACH MODEL
# ============================================================================

def get_target_layer(model, model_key):
    """Get the appropriate layer for Grad-CAM for each architecture"""
    
    if model_key == 'resnet50':
        return model.layer4[-1]  # Last residual block
    
    elif model_key == 'swin_tiny':
        # Last stage, last block, first norm layer
        return model.layers[-1].blocks[-1].norm1
    
    elif model_key == 'deit_small':
        # Last transformer block, first norm layer
        return model.blocks[-1].norm1
    
    else:
        raise ValueError(f"Unknown model: {model_key}")

# ============================================================================
# FUNCTION: COMPUTE GRAD-CAM FOR ONE MODEL
# ============================================================================

def compute_gradcam_for_model(model, model_key, val_loader_with_masks, device, num_samples=200):
    """
    Compute Grad-CAM and FAR for one model
    
    Args:
        model: Trained model
        model_key: 'resnet50', 'swin_tiny', or 'deit_small'
        val_loader_with_masks: DataLoader with original images and masks
        device: 'cuda' or 'cpu'
        num_samples: Number of validation samples to analyze (200 for speed)
    
    Returns:
        Dictionary with FAR statistics
    """
    
    print(f"\n{'─'*80}")
    print(f"Computing Grad-CAM for {model_key.upper()}")
    print(f"{'─'*80}")
    
    model.eval()
    
    # Get target layer
    target_layer = get_target_layer(model, model_key)
    
    # Create Grad-CAM extractor
    cam_extractor = GradCAM(model=model, target_layers=[target_layer], use_cuda=(device.type == 'cuda'))
    
    far_scores = []
    predictions_correct = 0
    total_samples = 0
    
    # Process validation samples
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader_with_masks, desc=f'Computing Grad-CAM', leave=False)):
            if i >= num_samples:
                break
            
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            masks = batch['mask'].numpy()[0]  # (224, 224)
            
            # Get model prediction
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            pred_class = predicted.item()
            
            # Track accuracy
            is_correct = (predicted.item() == labels.item())
            predictions_correct += is_correct
            total_samples += 1
            
            # Generate Grad-CAM
            try:
                targets = [ClassifierOutputTarget(pred_class)]
                grayscale_cam = cam_extractor(input_tensor=images, targets=targets)[0]
                
                # Normalize CAM to [0, 1]
                cam_min = grayscale_cam.min()
                cam_max = grayscale_cam.max()
                cam_normalized = (grayscale_cam - cam_min) / (cam_max - cam_min + 1e-8)
                
                # Compute FAR
                far = compute_far(cam_normalized, masks)
                far_scores.append(far)
                
            except Exception as e:
                print(f"  Warning: Grad-CAM failed for sample {i}: {e}")
                continue
    
    # Compute statistics
    if far_scores:
        far_mean = np.mean(far_scores)
        far_std = np.std(far_scores)
        far_median = np.median(far_scores)
        far_min = np.min(far_scores)
        far_max = np.max(far_scores)
    else:
        far_mean = far_std = far_median = far_min = far_max = 0.0
    
    accuracy = predictions_correct / total_samples if total_samples > 0 else 0.0
    
    print(f"\n  Samples analyzed: {len(far_scores)}")
    print(f"  Prediction accuracy: {accuracy*100:.2f}%")
    print(f"  Mean FAR: {far_mean:.4f} ± {far_std:.4f}")
    print(f"  Median FAR: {far_median:.4f}")
    print(f"  Range: [{far_min:.4f}, {far_max:.4f}]")
    
    # BDI is inverse of FAR (Background Dependence Index)
    bdi_mean = 1 - far_mean
    print(f"  Mean BDI: {bdi_mean:.4f}")
    
    return {
        'model_key': model_key,
        'far_mean': far_mean,
        'far_std': far_std,
        'far_median': far_median,
        'far_min': far_min,
        'far_max': far_max,
        'far_scores': far_scores,
        'bdi_mean': bdi_mean,
        'accuracy': accuracy,
        'samples_analyzed': len(far_scores)
    }

# ============================================================================
# CREATE VALIDATION LOADER WITH MASKS
# ============================================================================

print("\nPreparing validation dataset with segmentation masks...")

from torch.utils.data import Dataset, DataLoader

class ValidationDatasetWithMasks(Dataset):
    """Validation dataset that returns images with their segmentation masks"""
    
    def __init__(self, base_dataset, indices, transform=None):
        self.base_dataset = base_dataset
        self.indices = indices
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        actual_idx = self.indices[idx]
        image, mask = self.base_dataset[actual_idx]
        label = self.base_dataset._labels[actual_idx]
        
        # Resize to 224x224
        image = image.resize((224, 224))
        mask = mask.resize((224, 224))
        
        # Convert to numpy arrays
        image_np = np.array(image, dtype=np.float32) / 255.0
        mask_np = np.array(mask, dtype=np.uint8)
        
        # Binary mask (1=pet foreground, 0=background)
        binary_mask = (mask_np == 1).astype(np.float32)
        
        # Apply transform if provided
        if self.transform:
            image_np = self.transform(image_np)
        else:
            # Manual normalization
            IMAGENET_MEAN = [0.485, 0.456, 0.406]
            IMAGENET_STD = [0.229, 0.224, 0.225]
            image_np = (image_np - np.array(IMAGENET_MEAN)) / np.array(IMAGENET_STD)
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image_np.transpose(2, 0, 1)).float()
        
        return {
            'image': image_tensor,
            'mask': binary_mask,
            'label': label,
            'index': actual_idx
        }

# Create validation loader with masks
val_dataset_with_masks = ValidationDatasetWithMasks(
    dataset,
    val_indices,
    transform=None
)

val_loader_with_masks = DataLoader(
    val_dataset_with_masks,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"✓ Validation loader created: {len(val_dataset_with_masks)} images")

# ============================================================================
# LOAD TRAINED MODELS AND COMPUTE GRAD-CAM
# ============================================================================

all_gradcam_results = {}

for model_key in ['resnet50', 'swin_tiny', 'deit_small']:
    
    print(f"\n{'='*80}")
    print(f"LOADING {model_key.upper()} MODEL")
    print(f"{'='*80}")
    
    # Load model
    if model_key == 'resnet50':
        model = models.resnet50(weights=None)
        model.fc = nn.Linear(2048, 37)
    
    elif model_key == 'swin_tiny':
        model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=False)
        model.head = nn.Linear(768, 37)
    
    elif model_key == 'deit_small':
        model = timm.create_model('deit_small_patch16_224', pretrained=False)
        model.head = nn.Linear(384, 37)
    
    # Load checkpoint
    checkpoint_path = f'/kaggle/working/checkpoints/{model_key}/best.pth'
    
    try:
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✓ Model loaded from: {checkpoint_path}")
        print(f"  Best validation accuracy: {checkpoint.get('best_val_acc', 'N/A')}")
    except FileNotFoundError:
        print(f"✗ Checkpoint not found: {checkpoint_path}")
        print(f"  Skipping {model_key}...")
        continue
    
    model = model.to(device).eval()
    
    # Compute Grad-CAM
    gradcam_result = compute_gradcam_for_model(
        model,
        model_key,
        val_loader_with_masks,
        device,
        num_samples=200
    )
    
    all_gradcam_results[model_key] = gradcam_result
    
    # Clear GPU memory
    torch.cuda.empty_cache()

# ============================================================================
# CREATE RESULTS TABLE
# ============================================================================

print("\n" + "="*80)
print("GRAD-CAM ANALYSIS RESULTS - ALL MODELS")
print("="*80)

results_table = []

for model_key, result in all_gradcam_results.items():
    results_table.append({
        'Model': model_key.replace('_', '-').upper(),
        'FAR (Mean±Std)': f"{result['far_mean']:.4f}±{result['far_std']:.4f}",
        'BDI': f"{result['bdi_mean']:.4f}",
        'Median FAR': f"{result['far_median']:.4f}",
        'Range [min, max]': f"[{result['far_min']:.4f}, {result['far_max']:.4f}]",
        'Accuracy (%)': f"{result['accuracy']*100:.2f}",
        'Samples': result['samples_analyzed']
    })

results_df = pd.DataFrame(results_table)

print("\n" + results_df.to_string(index=False))

# Save results
os.makedirs('/kaggle/working/results', exist_ok=True)
results_df.to_csv('/kaggle/working/results/gradcam_far_all_models.csv', index=False)

print("\n✓ Results saved to: /kaggle/working/results/gradcam_far_all_models.csv")

# ============================================================================
# INTERPRETATION
# ============================================================================

print("\n" + "="*80)
print("INTERPRETATION")
print("="*80)

for model_key, result in all_gradcam_results.items():
    print(f"\n{result['model_key'].upper()}:")
    print(f"  FAR = {result['far_mean']:.4f}")
    print(f"  → Model focuses {result['far_mean']*100:.1f}% of attention on pet foreground")
    print(f"  → Model focuses {result['bdi_mean']*100:.1f}% of attention on background")
    
    if result['far_mean'] > 0.65:
        print(f"  → ✓ High foreground focus (good sign!)")
    else:
        print(f"  → ⚠ Relatively low foreground focus")

print("\n" + "="*80)
print("✅ GRAD-CAM ANALYSIS COMPLETE")
print("="*80)
print("\nNext step: Evaluate models on COUNTERFACTUAL images to test if background")
print("information actually influences predictions (compute ∆Acc)")
